In [1]:
!pip install transformers datasets peft accelerate bitsandbytes evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install --upgrade peft transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu1

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

In [4]:
MODEL_NAME = "distilbert-base-uncased"  # replace with your base model
TASK = "sst2"  # sample task from GLUE (binary sentiment)
MAX_LEN = 128
BATCH_SIZE = 16
LR = 2e-4
EPOCHS = 3
OUTPUT_DIR = "./lora_model"

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
dataset = load_dataset("glue", TASK)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tokenize(batch):
    return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=MAX_LEN)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
# --- Load base model ---
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [9]:
for name, module in model.named_modules():
    print(name)



distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
distilbert.transformer.layer.1.attention.q

In [10]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_lin","v_lin"],  # correct for DistilBERT
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)


In [11]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # shows only LoRA params are trainable

trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


In [12]:
# --- Training ---
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=2,
    fp16=True if device=="cuda" else False,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.237000,0.269761,0.894495
2,0.193800,0.314147,0.897936
3,0.122600,0.345756,0.899083


TrainOutput(global_step=12630, training_loss=0.21770143307869338, metrics={'train_runtime': 580.7249, 'train_samples_per_second': 347.922, 'train_steps_per_second': 21.749, 'total_flos': 6828804290442240.0, 'train_loss': 0.21770143307869338, 'epoch': 3.0})

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define label mapping
label2id = {"negative": 0, "positive": 1}  # or spam example {"not spam":0, "spam":1}
id2label = {v: k for k, v in label2id.items()}

# Load your LoRA-adapted model
model = AutoModelForSequenceClassification.from_pretrained("Noobhacker69/Low-Rank_Adaptation_NLP_Classifier")

# Set the labels in the model config
model.config.id2label = id2label
model.config.label2id = label2id

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Noobhacker69/Low-Rank_Adaptation_NLP_Classifier")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [37]:
model.config.id2label = {0: "negative", 1: "positive"}
model.config.label2id = {"negative": 0, "positive": 1}

# Save LoRA-adapted model
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("LoRA fine-tuning complete! Saved at:", OUTPUT_DIR)

LoRA fine-tuning complete! Saved at: ./lora_model


In [38]:
model.push_to_hub("Noobhacker69/Low-Rank_Adaptation_NLP_Classifier")
tokenizer.push_to_hub("Noobhacker69/Low-Rank_Adaptation_NLP_Classifier")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########| 3.55MB / 3.55MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Noobhacker69/Low-Rank_Adaptation_NLP_Classifier/commit/ae6f28d5a78d961f456ac7bfd3fb4a9ecb7498ca', commit_message='Upload tokenizer', commit_description='', oid='ae6f28d5a78d961f456ac7bfd3fb4a9ecb7498ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Noobhacker69/Low-Rank_Adaptation_NLP_Classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Noobhacker69/Low-Rank_Adaptation_NLP_Classifier'), pr_revision=None, pr_num=None)

In [33]:
from huggingface_hub import notebook_login
notebook_login()

In [34]:
# Push to Hub (your username/repo_name)
repo_name = "Noobhacker69/Low-Rank_Adaptation_NLP_Classifier"  # you choose the name
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########| 3.55MB / 3.55MB            

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Noobhacker69/Low-Rank_Adaptation_NLP_Classifier/commit/ae6f28d5a78d961f456ac7bfd3fb4a9ecb7498ca', commit_message='Upload tokenizer', commit_description='', oid='ae6f28d5a78d961f456ac7bfd3fb4a9ecb7498ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Noobhacker69/Low-Rank_Adaptation_NLP_Classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='Noobhacker69/Low-Rank_Adaptation_NLP_Classifier'), pr_revision=None, pr_num=None)

In [43]:
from transformers import pipeline
classifier = pipeline("text-classification", model="Noobhacker69/Low-Rank_Adaptation_NLP_Classifier")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [44]:
print(classifier("I love Hugging Face!"))

[{'label': 'LABEL_1', 'score': 0.9992761015892029}]
